<a href="https://colab.research.google.com/github/aaronbbarclay/mine/blob/master/Macbeth_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using mask RCNN to train macbeth detection

https://github.com/matterport/Mask_RCNN/

In [1]:
!pip show tensorflow

Name: tensorflow
Version: 1.13.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python2.7/dist-packages
Requires: tensorflow-estimator, grpcio, mock, keras-applications, enum34, protobuf, keras-preprocessing, gast, tensorboard, wheel, absl-py, backports.weakref, six, numpy, termcolor, astor
Required-by: magenta, fancyimpute


In [8]:
%cd
!git clone --quiet https://github.com/tensorflow/models.git
!git clone --quiet https://github.com/matterport/Mask_RCNN/
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

/root
fatal: destination path 'models' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pil is already the newest version (5.1.0-1).
python-tk is already the newest version (2.7.15~rc1-1).
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-lxml is already the newest version (4.2.1-1ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [17]:
%cd /root/Mask_RCNN/
%ls

/root/Mask_RCNN
assets/  LICENSE      mrcnn/     requirements.txt  setup.cfg
images/  MANIFEST.in  README.md  samples/          setup.py


In [0]:
# Install pycotools
!pip install -q pycocotools

In [10]:
# Compile protocol buffers
%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.

/root/models/research


In [0]:
# Configure env variables
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/Mask_RCNN/'

import mrcnn

In [0]:
import six
import mrcnn

In [21]:
# Check environment variables
print(os.environ['PYTHONPATH'])

/env/python:/root/models/research/:/root/models/research/slim/:/root/models/research/:/root/models/research/slim/:/root/Mask_RCNN/mrcnn/:/root/models/research/:/root/models/research/slim/:/root/Mask_RCNN/:/root/models/research/:/root/models/research/slim/:/root/Mask_RCNN/


In [22]:
# Run tests
!python ~/models/research/object_detection/builders/model_builder_test.py


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.079s

OK (skipped=1)


In [23]:
# Create directories, get my repo for the images
%mkdir ~/data
%cd ~/data

!git clone https://github.com/aaronbbarclay/mine
%cd mine/machineLearning/macbethIdentify/data/
%ls
!unzip macbeth_identify.zip

/root/data
Cloning into 'mine'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 83 (delta 47), reused 9 (delta 0), pack-reused 0
Unpacking objects: 100% (83/83), done.
/root/data/mine/machineLearning/macbethIdentify/data
macbeth_identify.zip*
Archive:  macbeth_identify.zip
   creating: macbeth_identify/
  inflating: macbeth_identify/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/macbeth_identify/
  inflating: __MACOSX/macbeth_identify/._.DS_Store  
  inflating: macbeth_identify/label_map.pbtxt  
  inflating: __MACOSX/macbeth_identify/._label_map.pbtxt  
  inflating: macbeth_identify/train.txt  
  inflating: __MACOSX/macbeth_identify/._train.txt  
   creating: macbeth_identify/img/
  inflating: macbeth_identify/img/IMG_1336.jpg  
   creating: __MACOSX/macbeth_identify/img/
  inflating: __MACOSX/macbeth_identify/img/._IMG_1336.jpg  
  inflating: macbeth_identify/img/IMG_1322.jp

In [24]:
# Setup some stuff
%cd ~/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/
%pwd
%ls
imageFilesBasePath = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/img'
annotationsFilesBasePath = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/annotations/'
labelMapPath = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/label_map.pbtxt'
testPath = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/test.txt'
trainPath = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/train.txt'
valPath = '/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify/val.txt'


/root/data/mine/machineLearning/macbethIdentify/data/macbeth_identify
annotations/  img/  label_map.pbtxt*  test.txt*  train.txt*  val.txt*


In [25]:
# Update and pull repo if required
%cd ~/data/mine/
%ls
!git pull


/root/data/mine
inference.ipynb  machineLearning/      README.md
Macbeth1.ipynb   mask_rcnn_demo.ipynb
Already up to date.


In [0]:
%cd /root/data/
%mkdir trimaps

from PIL import Image
image = Image.new("RGB", (640, 480))

for filename in os.listdir(annotationsFilesBasePath):
  filenameTokens = filename.split(".")
  newFileName = "trimaps/" + filename + '.png'
  print('Saving: {0}'.format(newFileName))
  image.save(newFileName)

In [0]:

from mrcnn.config import Config

# Derived from https://github.com/matterport/Mask_RCNN/blob/master/samples/shapes/train_shapes.ipynb
class MacbethConfig(Config):
    # Give the configuration a recognizable name
    NAME = "macbeth"
    
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8
    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 1
    
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 128
    IMAGE_MAX_DIM = 128
    
    # Use smaller anchors because our image and objects are small
    RPM_ANCHOR_SCALES = (8, 16, 32, 64, 128)
    
    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32
    
    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100
    
    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = MacbethConfig()
config.display()

In [0]:
import 

# URL from which to download the latest COCO trained weights
COCO_MODEL_URL = "https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5"

class MacbethDataset(mrcnn.utils.Dataset):
    
    


